In [ ]:
import os, sys
sys.path.append('..')

NOTION_DATABASE_ID = "571b189f4a4a4125b39e5da1f5b59583"

from notion_client import Client

notion = Client(auth=os.environ["NOTION_API_KEY"])

In [ ]:
my_page = notion.databases.query(
    **{
        "database_id": NOTION_DATABASE_ID,
        "filter": {
            "property": "股票代码",
            "rich_text": {
                "equals": "601919",
            },
        },
    }
)
my_page

In [ ]:
def query_database(database_id, property_name, property_value):
    """
    查询数据库中是否存在指定属性值的记录
    :param database_id: 数据库ID
    :param property_name: 属性名称
    :param property_value: 属性值
    :return: 如果存在返回page_id，否则返回None
    """
    filter_params = {
        "property": property_name,
        "rich_text": {
            "equals": property_value
        }
    }
    print(filter_params)
    response = notion.databases.query(
        **{
            "database_id": database_id,
            "filter": filter_params
        }
    )

    if response['results']:
        return response['results'][0]['id']
    else:
        return None

def create_page_in_database(database_id, title, property_name, property_value):
    """
    在数据库中创建一行新的记录
    :param database_id: 数据库ID
    :param title: 记录的标题
    :param property_name: 属性名称
    :param property_value: 属性值
    :return: 新创建的page_id
    """
    new_page = {
        "parent": {"database_id": database_id},
        "properties": {
            "Title": {
                "title": [
                    {
                        "text": {
                            "content": title
                        }
                    }
                ]
            },
            property_name: {
                "rich_text": [
                    {
                        "text": {
                            "content": property_value
                        }
                    }
                ]
            }
        }
    }

    response = notion.pages.create(**new_page)
    return response['id']

def add_heading_to_page(page_id, heading_text):
    """
    在指定页面中增加一个Heading 2
    :param page_id: 页面ID
    :param heading_text: Heading的文本内容
    """
    heading_block = {
        "parent": {"page_id": page_id},
        "properties": {},
        "children": [
            {
                "object": "block",
                "type": "heading_2",
                "heading_2": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": heading_text
                            }
                        }
                    ]
                }
            }
        ]
    }
    response = notion.blocks.children.append(
        block_id=page_id,
        children=heading_block["children"]
    )
    return response

In [ ]:
property_name = "股票代码"
property_value = "601919"
title = "中远海控"

page_id = query_database(NOTION_DATABASE_ID, property_name, property_value)
if not page_id:
    print(f"不存在{property_name}为{property_value}的记录。")
    page_id = create_page_in_database(NOTION_DATABASE_ID, title, property_name, property_value)
    print(f"已创建新的记录：{title}，{property_name}为{property_value}，页面ID为{page_id}。")

print(f"存在{property_name}为{property_value}的记录，页面ID为{page_id}。")
# 添加Heading 2到页面
# heading_text = "经营分析"
# add_heading_to_page(page_id, heading_text)


In [ ]:
response = notion.blocks.children.append(
    block_id=page_id,
    children=[
        {
            "object": "block",
            "type": "heading_1",
            "heading_1": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {
                            "content": "这是一个标题"
                        }
                    }
                ]
            }
        },
        {
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {
                            "content": "这是一段文字内容。"
                        }
                    }
                ]
            }
        }
    ]
)

print("Blocks added successfully:", response)

In [ ]:
from pprint import pprint
from utils.notion_helper import array_to_notion_blocks, add_blocks_after_heading
from utils.str_helper import html_table_to_array

heading_text = "经营分析"
html_table = """
<table class="table"><tr><th style="width: 10%;">2023-12-31</th><th>主营构成</th><th style="width: 12%;">主营收入(元)</th><th style="width: 8%;">收入比例</th><th style="width: 12%;">主营成本(元)</th><th style="width: 8%;">成本比例</th><th style="width: 12%;">主营利润(元)</th><th style="width: 8%;">利润比例</th><th style="width: 9%;">毛利率(%)</th></tr><tr><td rowspan="2" class="white"> 按行业分类</td><td class="white">智能硬件</td><td class="content">86.39亿</td><td class="content">99.83%</td><td class="content">38.60亿</td><td class="content">99.42%</td><td class="content">47.80亿</td><td class="content">100.18%</td><td class="content">55.32%</td></tr><tr><!----><td class="white">其他(补充)</td><td class="content">1430万</td><td class="content">0.17%</td><td class="content">2270万</td><td class="content">0.58%</td><td class="content">-839.3万</td><td class="content">-0.18%</td><td class="content">-58.68%</td></tr><tr><td rowspan="3" class="white"> 按产品分类</td><td class="white">智能扫地机及配件</td><td class="content">80.85亿</td><td class="content">93.43%</td><td class="content">35.46亿</td><td class="content">91.32%</td><td class="content">45.40亿</td><td class="content">95.15%</td><td class="content">56.15%</td></tr><tr><!----><td class="white">其他智能电器产品</td><td class="content">5.541亿</td><td class="content">6.40%</td><td class="content">3.141亿</td><td class="content">8.09%</td><td class="content">2.400亿</td><td class="content">5.03%</td><td class="content">43.31%</td></tr><tr><!----><td class="white">其他(补充)</td><td class="content">1430万</td><td class="content">0.17%</td><td class="content">2270万</td><td class="content">0.58%</td><td class="content">-839.3万</td><td class="content">-0.18%</td><td class="content">-58.68%</td></tr><tr><td rowspan="3" class="white"> 按地区分类</td><td class="white">境内</td><td class="content">44.11亿</td><td class="content">50.97%</td><td class="content">22.38亿</td><td class="content">57.64%</td><td class="content">21.73亿</td><td class="content">45.54%</td><td class="content">49.26%</td></tr><tr><!----><td class="white">境外</td><td class="content">42.29亿</td><td class="content">48.87%</td><td class="content">16.22亿</td><td class="content">41.77%</td><td class="content">26.07亿</td><td class="content">54.64%</td><td class="content">61.65%</td></tr><tr><!----><td class="white">其他(补充)</td><td class="content">1430万</td><td class="content">0.17%</td><td class="content">2270万</td><td class="content">0.58%</td><td class="content">-839.3万</td><td class="content">-0.18%</td><td class="content">-58.68%</td></tr></table>
"""

array = html_table_to_array(html_table)
table_blocks = array_to_notion_blocks(array)
add_blocks_after_heading(page_id, heading_text, [table_blocks])